In [22]:
import pandas as pd
import mne as mne
import os 
import time
import numpy as np
import matplotlib.pyplot as plt
import joblib
import constants
from IPython.utils import io
import time
import sys
import yasa
from scipy.signal import welch

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GroupShuffleSplit


from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier

#Import my modules
import format_eeg_data
import constants
import eeg_stat_ts
import run_expts

In [57]:
# Set display options to show all rows and columns
pd.set_option('display.max_rows', 50)  # Show rows
pd.set_option('display.max_columns', 160)  # Show columns

In [43]:
core_path = '/user/home/ko20929/work/RBD_using_custom_package/Execute New Experiments/Baseline_Extensions/Gen_New_Features/generated_feats/'
data_types = ['N2', 'N3','REM', 'Wake', 'N1']
data_type = 'Wake' # Select data_type to load

load_path = core_path + data_type

X = pd.read_hdf(load_path + 'six_second_max_freq_stats_df.h5', key='df', mode='r')
y = pd.read_hdf(load_path + '_y.h5', key='df', mode='r') 
groups = pd.read_hdf(load_path + '_groups.h5', key='df', mode='r')  

# Replace time sereis data with mean of the data ______________________________________________________________________________________________________

# Defining a function to replace time series of values with their mean
def function(x):
    return x.values.mean()
    
#Construct the static features
static_features_df = X.apply(np.vectorize(function))
X = static_features_df.copy()

In [15]:
#Generate region to features dictionary to enable experiments to be run regionally
regional_features_dict = {}
region_channel_dict = constants.region_to_channel_dict
regions = list(region_channel_dict.keys())
for region in regions:
    region_features = [col for col in X.columns if '_' + region in col]
    if len(region_features) > 0 : 
        regional_features_dict[region] = region_features

In [50]:
#Create the combined regions dictionary
regions = list(regional_features_dict.keys())
combined_regions_features_dict = {}
    
for i, region_1 in enumerate(regions):
    for region_2 in regions[i+1:]:
        new_key = region_1 + '_' + region_2
        combined_regions_features_dict[new_key] = regional_features_dict[region_1] + regional_features_dict[region_2]

## Expt 1

### Regional

In [23]:
#Now X, Y and groups have been loaded for N1 data - generate X,y groups for a specific experiment
#1.Generate expt specific X,y,groups
t1 = time.time()

X_expt , y_expt , groups_expt, expt_info = run_expts.generate_expt_x_y_groups(X,y,groups, 1 )
results_df = run_expts.run_mv_tsc(X_expt,y_expt,groups_expt, {'RF' : RandomForestClassifier(random_state = 5) , 'DT' : DecisionTreeClassifier() , 'Ada_B' : AdaBoostClassifier(random_state = 5)} , return_df = True , subset_names_and_cols = regional_features_dict, random_states = [1,2] )
res_df = run_expts.generate_subset_acc_std(results_df, return_df = True)

display(pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'RF' in col]].mean(axis = 0)).T)
display(pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'DT' in col]].mean(axis = 0)).T)
display(pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'Ada_B' in col]].mean(axis = 0)).T)


t2 = time.time()
time_taken = t2-t1
print(time_taken)

,Prefrontal_RF_acc,Frontal_RF_acc,Left Frontal_RF_acc,Right Frontal_RF_acc,Central_RF_acc,Left Central_RF_acc,Right Central_RF_acc,Left Temporal_RF_acc,Right Temporal_RF_acc,Parietal_RF_acc,Left Parietal_RF_acc,Right Parietal_RF_acc,Occipital_RF_acc
0,0.65,0.483333,0.633333,0.583333,0.5,0.483333,0.383333,0.5,0.4,0.4,0.566667,0.55,0.55


,Prefrontal_DT_acc,Frontal_DT_acc,Left Frontal_DT_acc,Right Frontal_DT_acc,Central_DT_acc,Left Central_DT_acc,Right Central_DT_acc,Left Temporal_DT_acc,Right Temporal_DT_acc,Parietal_DT_acc,Left Parietal_DT_acc,Right Parietal_DT_acc,Occipital_DT_acc
0,0.716667,0.483333,0.55,0.5,0.5,0.55,0.416667,0.466667,0.45,0.466667,0.55,0.516667,0.466667


,Prefrontal_Ada_B_acc,Frontal_Ada_B_acc,Left Frontal_Ada_B_acc,Right Frontal_Ada_B_acc,Central_Ada_B_acc,Left Central_Ada_B_acc,Right Central_Ada_B_acc,Left Temporal_Ada_B_acc,Right Temporal_Ada_B_acc,Parietal_Ada_B_acc,Left Parietal_Ada_B_acc,Right Parietal_Ada_B_acc,Occipital_Ada_B_acc
0,0.75,0.5,0.65,0.533333,0.483333,0.466667,0.533333,0.566667,0.416667,0.483333,0.516667,0.616667,0.45


43.95807409286499


### Combined Regions

In [58]:
#Now X, Y and groups have been loaded for N1 data - generate X,y groups for a specific experiment
#1.Generate expt specific X,y,groups
t1 = time.time()

X_expt , y_expt , groups_expt, expt_info = run_expts.generate_expt_x_y_groups(X,y,groups, 1 )
results_df = run_expts.run_mv_tsc(X_expt,y_expt,groups_expt, {'RF' : RandomForestClassifier(random_state = 5) , 'DT' : DecisionTreeClassifier() , 'Ada_B' : AdaBoostClassifier(random_state = 5)} , return_df = True , subset_names_and_cols = combined_regions_features_dict, random_states = [1,2] )
res_df = run_expts.generate_subset_acc_std(results_df, return_df = True)

display(pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'RF' in col]].mean(axis = 0)).T)
display(pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'DT' in col]].mean(axis = 0)).T)
display(pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'Ada_B' in col]].mean(axis = 0)).T)


t2 = time.time()
time_taken = t2-t1
print(time_taken)

,Prefrontal_Frontal_RF_acc,Prefrontal_Left Frontal_RF_acc,Prefrontal_Right Frontal_RF_acc,Prefrontal_Central_RF_acc,Prefrontal_Left Central_RF_acc,Prefrontal_Right Central_RF_acc,Prefrontal_Left Temporal_RF_acc,Prefrontal_Right Temporal_RF_acc,Prefrontal_Parietal_RF_acc,Prefrontal_Left Parietal_RF_acc,Prefrontal_Right Parietal_RF_acc,Prefrontal_Occipital_RF_acc,Frontal_Left Frontal_RF_acc,Frontal_Right Frontal_RF_acc,Frontal_Central_RF_acc,Frontal_Left Central_RF_acc,Frontal_Right Central_RF_acc,Frontal_Left Temporal_RF_acc,Frontal_Right Temporal_RF_acc,Frontal_Parietal_RF_acc,Frontal_Left Parietal_RF_acc,Frontal_Right Parietal_RF_acc,Frontal_Occipital_RF_acc,Left Frontal_Right Frontal_RF_acc,Left Frontal_Central_RF_acc,Left Frontal_Left Central_RF_acc,Left Frontal_Right Central_RF_acc,Left Frontal_Left Temporal_RF_acc,Left Frontal_Right Temporal_RF_acc,Left Frontal_Parietal_RF_acc,Left Frontal_Left Parietal_RF_acc,Left Frontal_Right Parietal_RF_acc,Left Frontal_Occipital_RF_acc,Right Frontal_Central_RF_acc,Right Frontal_Left Central_RF_acc,Right Frontal_Right Central_RF_acc,Right Frontal_Left Temporal_RF_acc,Right Frontal_Right Temporal_RF_acc,Right Frontal_Parietal_RF_acc,Right Frontal_Left Parietal_RF_acc,Right Frontal_Right Parietal_RF_acc,Right Frontal_Occipital_RF_acc,Central_Left Central_RF_acc,Central_Right Central_RF_acc,Central_Left Temporal_RF_acc,Central_Right Temporal_RF_acc,Central_Parietal_RF_acc,Central_Left Parietal_RF_acc,Central_Right Parietal_RF_acc,Central_Occipital_RF_acc,Left Central_Right Central_RF_acc,Left Central_Left Temporal_RF_acc,Left Central_Right Temporal_RF_acc,Left Central_Parietal_RF_acc,Left Central_Left Parietal_RF_acc,Left Central_Right Parietal_RF_acc,Left Central_Occipital_RF_acc,Right Central_Left Temporal_RF_acc,Right Central_Right Temporal_RF_acc,Right Central_Parietal_RF_acc,Right Central_Left Parietal_RF_acc,Right Central_Right Parietal_RF_acc,Right Central_Occipital_RF_acc,Left Temporal_Right Temporal_RF_acc,Left Temporal_Parietal_RF_acc,Left Temporal_Left Parietal_RF_acc,Left Temporal_Right Parietal_RF_acc,Left Temporal_Occipital_RF_acc,Right Temporal_Parietal_RF_acc,Right Temporal_Left Parietal_RF_acc,Right Temporal_Right Parietal_RF_acc,Right Temporal_Occipital_RF_acc,Parietal_Left Parietal_RF_acc,Parietal_Right Parietal_RF_acc,Parietal_Occipital_RF_acc,Left Parietal_Right Parietal_RF_acc,Left Parietal_Occipital_RF_acc,Right Parietal_Occipital_RF_acc
0,0.633333,0.666667,0.633333,0.65,0.683333,0.6,0.65,0.566667,0.583333,0.6,0.633333,0.65,0.583333,0.483333,0.516667,0.45,0.433333,0.433333,0.416667,0.466667,0.4,0.516667,0.466667,0.6,0.583333,0.6,0.616667,0.5,0.566667,0.55,0.583333,0.65,0.583333,0.5,0.433333,0.45,0.466667,0.433333,0.466667,0.5,0.533333,0.5,0.533333,0.416667,0.55,0.483333,0.466667,0.55,0.566667,0.45,0.483333,0.483333,0.5,0.4,0.516667,0.516667,0.466667,0.45,0.4,0.4,0.466667,0.416667,0.4,0.383333,0.466667,0.483333,0.6,0.45,0.416667,0.416667,0.483333,0.383333,0.483333,0.416667,0.433333,0.5,0.433333,0.466667


,Prefrontal_Frontal_DT_acc,Prefrontal_Left Frontal_DT_acc,Prefrontal_Right Frontal_DT_acc,Prefrontal_Central_DT_acc,Prefrontal_Left Central_DT_acc,Prefrontal_Right Central_DT_acc,Prefrontal_Left Temporal_DT_acc,Prefrontal_Right Temporal_DT_acc,Prefrontal_Parietal_DT_acc,Prefrontal_Left Parietal_DT_acc,Prefrontal_Right Parietal_DT_acc,Prefrontal_Occipital_DT_acc,Frontal_Left Frontal_DT_acc,Frontal_Right Frontal_DT_acc,Frontal_Central_DT_acc,Frontal_Left Central_DT_acc,Frontal_Right Central_DT_acc,Frontal_Left Temporal_DT_acc,Frontal_Right Temporal_DT_acc,Frontal_Parietal_DT_acc,Frontal_Left Parietal_DT_acc,Frontal_Right Parietal_DT_acc,Frontal_Occipital_DT_acc,Left Frontal_Right Frontal_DT_acc,Left Frontal_Central_DT_acc,Left Frontal_Left Central_DT_acc,Left Frontal_Right Central_DT_acc,Left Frontal_Left Temporal_DT_acc,Left Frontal_Right Temporal_DT_acc,Left Frontal_Parietal_DT_acc,Left Frontal_Left Parietal_DT_acc,Left Frontal_Right Parietal_DT_acc,Left Frontal_Occipital_DT_acc,Right Frontal_Central_DT_acc,Right Frontal_Left Central_DT_acc,Right Frontal_Right Central_DT_acc,Right Frontal_Left Temporal_DT_acc,Right Frontal_Right Temporal_DT_acc,Right Frontal_Parietal_DT_acc,Right Frontal_Left Parietal_DT_acc,Right Frontal_Right Parietal_DT_acc,Right Frontal_Occipital_DT_acc,Central_Left Central_DT_acc,Central_Right Central_DT_acc,Central_Left Temporal_DT_acc,Central_Right Temporal_DT_acc,Central_Parietal_DT_acc,Central_Left Parietal_DT_acc,Central_Right Parietal_DT_acc,Central_Occipital_DT_acc,Left Central_Right Central_DT_acc,Left Central_Left Temporal_DT_acc,Left Central_Right Temporal_DT_acc,Left Central_Parietal_DT_acc,Left Central_Left Parietal_DT_acc,Left Central_Right Parietal_DT_acc,Left Central_Occipital_DT_acc,Right Central_Left Temporal_DT_acc,Right Central_Right Temporal_DT_acc,Right Central_Parietal_DT_acc,Right Central_Left Parietal_DT_acc,Right Central_Right Parietal_DT_acc,Right Central_Occipital_DT_acc,Left Temporal_Right Temporal_DT_acc,Left Temporal_Parietal_DT_acc,Left Temporal_Left Parietal_DT_acc,Left Temporal_Right Parietal_DT_acc,Left Temporal_Occipital_DT_acc,Right Temporal_Parietal_DT_acc,Right Temporal_Left Parietal_DT_acc,Right Temporal_Right Parietal_DT_acc,Right Temporal_Occipital_DT_acc,Parietal_Left Parietal_DT_acc,Parietal_Right Parietal_DT_acc,Parietal_Occipital_DT_acc,Left Parietal_Right Parietal_DT_acc,Left Parietal_Occipital_DT_acc,Right Parietal_Occipital_DT_acc
0,0.75,0.616667,0.716667,0.633333,0.666667,0.633333,0.683333,0.616667,0.65,0.666667,0.733333,0.75,0.583333,0.45,0.466667,0.483333,0.5,0.45,0.5,0.4,0.433333,0.533333,0.55,0.583333,0.516667,0.583333,0.516667,0.5,0.433333,0.55,0.5,0.533333,0.5,0.5,0.516667,0.533333,0.466667,0.516667,0.4,0.466667,0.566667,0.533333,0.466667,0.433333,0.466667,0.416667,0.45,0.466667,0.516667,0.483333,0.5,0.583333,0.5,0.4,0.45,0.5,0.35,0.383333,0.333333,0.433333,0.366667,0.4,0.483333,0.483333,0.433333,0.4,0.466667,0.383333,0.433333,0.45,0.516667,0.416667,0.433333,0.483333,0.4,0.5,0.383333,0.516667


,Prefrontal_Frontal_Ada_B_acc,Prefrontal_Left Frontal_Ada_B_acc,Prefrontal_Right Frontal_Ada_B_acc,Prefrontal_Central_Ada_B_acc,Prefrontal_Left Central_Ada_B_acc,Prefrontal_Right Central_Ada_B_acc,Prefrontal_Left Temporal_Ada_B_acc,Prefrontal_Right Temporal_Ada_B_acc,Prefrontal_Parietal_Ada_B_acc,Prefrontal_Left Parietal_Ada_B_acc,Prefrontal_Right Parietal_Ada_B_acc,Prefrontal_Occipital_Ada_B_acc,Frontal_Left Frontal_Ada_B_acc,Frontal_Right Frontal_Ada_B_acc,Frontal_Central_Ada_B_acc,Frontal_Left Central_Ada_B_acc,Frontal_Right Central_Ada_B_acc,Frontal_Left Temporal_Ada_B_acc,Frontal_Right Temporal_Ada_B_acc,Frontal_Parietal_Ada_B_acc,Frontal_Left Parietal_Ada_B_acc,Frontal_Right Parietal_Ada_B_acc,Frontal_Occipital_Ada_B_acc,Left Frontal_Right Frontal_Ada_B_acc,Left Frontal_Central_Ada_B_acc,Left Frontal_Left Central_Ada_B_acc,Left Frontal_Right Central_Ada_B_acc,Left Frontal_Left Temporal_Ada_B_acc,Left Frontal_Right Temporal_Ada_B_acc,Left Frontal_Parietal_Ada_B_acc,Left Frontal_Left Parietal_Ada_B_acc,Left Frontal_Right Parietal_Ada_B_acc,Left Frontal_Occipital_Ada_B_acc,Right Frontal_Central_Ada_B_acc,Right Frontal_Left Central_Ada_B_acc,Right Frontal_Right Central_Ada_B_acc,Right Frontal_Left Temporal_Ada_B_acc,Right Frontal_Right Temporal_Ada_B_acc,Right Frontal_Parietal_Ada_B_acc,Right Frontal_Left Parietal_Ada_B_acc,Right Frontal_Right Parietal_Ada_B_acc,Right Frontal_Occipital_Ada_B_acc,Central_Left Central_Ada_B_acc,Central_Right Central_Ada_B_acc,Central_Left Temporal_Ada_B_acc,Central_Right Temporal_Ada_B_acc,Central_Parietal_Ada_B_acc,Central_Left Parietal_Ada_B_acc,Central_Right Parietal_Ada_B_acc,Central_Occipital_Ada_B_acc,Left Central_Right Central_Ada_B_acc,Left Central_Left Temporal_Ada_B_acc,Left Central_Right Temporal_Ada_B_acc,Left Central_Parietal_Ada_B_acc,Left Central_Left Parietal_Ada_B_acc,Left Central_Right Parietal_Ada_B_acc,Left Central_Occipital_Ada_B_acc,Right Central_Left Temporal_Ada_B_acc,Right Central_Right Temporal_Ada_B_acc,Right Central_Parietal_Ada_B_acc,Right Central_Left Parietal_Ada_B_acc,Right Central_Right Parietal_Ada_B_acc,Right Central_Occipital_Ada_B_acc,Left Temporal_Right Temporal_Ada_B_acc,Left Temporal_Parietal_Ada_B_acc,Left Temporal_Left Parietal_Ada_B_acc,Left Temporal_Right Parietal_Ada_B_acc,Left Temporal_Occipital_Ada_B_acc,Right Temporal_Parietal_Ada_B_acc,Right Temporal_Left Parietal_Ada_B_acc,Right Temporal_Right Parietal_Ada_B_acc,Right Temporal_Occipital_Ada_B_acc,Parietal_Left Parietal_Ada_B_acc,Parietal_Right Parietal_Ada_B_acc,Parietal_Occipital_Ada_B_acc,Left Parietal_Right Parietal_Ada_B_acc,Left Parietal_Occipital_Ada_B_acc,Right Parietal_Occipital_Ada_B_acc
0,0.666667,0.65,0.666667,0.583333,0.566667,0.65,0.733333,0.566667,0.6,0.616667,0.6,0.583333,0.583333,0.533333,0.45,0.5,0.516667,0.533333,0.466667,0.45,0.4,0.466667,0.516667,0.65,0.466667,0.6,0.516667,0.566667,0.516667,0.6,0.55,0.633333,0.583333,0.516667,0.583333,0.55,0.5,0.516667,0.483333,0.483333,0.6,0.433333,0.5,0.483333,0.583333,0.433333,0.483333,0.516667,0.5,0.433333,0.533333,0.566667,0.433333,0.416667,0.45,0.566667,0.466667,0.516667,0.483333,0.383333,0.45,0.533333,0.433333,0.416667,0.466667,0.566667,0.533333,0.416667,0.416667,0.4,0.45,0.366667,0.433333,0.533333,0.483333,0.516667,0.483333,0.616667


114.3186297416687


### Using all of the data 

In [44]:
all_data_dict = {'All_regions' : list(X.columns) , 'All_regions_2' : list(X.columns)}
# all_data_dict

In [45]:
#Now X, Y and groups have been loaded for N1 data - generate X,y groups for a specific experiment
#1.Generate expt specific X,y,groups
t1 = time.time()

X_expt , y_expt , groups_expt, expt_info = run_expts.generate_expt_x_y_groups(X,y,groups, 1 )
results_df = run_expts.run_mv_tsc(X_expt,y_expt,groups_expt, {'RF' : RandomForestClassifier(random_state = 5) , 'DT' : DecisionTreeClassifier() , 'Ada_B' : AdaBoostClassifier(random_state = 5)} , return_df = True , subset_names_and_cols = all_data_dict, random_states = [1,2] )
res_df = run_expts.generate_subset_acc_std(results_df, return_df = True)

display(pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'RF' in col]].mean(axis = 0)).T)
display(pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'DT' in col]].mean(axis = 0)).T)
display(pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'Ada_B' in col]].mean(axis = 0)).T)


t2 = time.time()
time_taken = t2-t1
print(time_taken)

,All_regions_RF_acc,All_regions_2_RF_acc
0,0.533333,0.533333


,All_regions_DT_acc,All_regions_2_DT_acc
0,0.566667,0.566667


,All_regions_Ada_B_acc,All_regions_2_Ada_B_acc
0,0.583333,0.583333


3.4565749168395996


## Expt 2

### Regional

In [24]:
#Now X, Y and groups have been loaded for N1 data - generate X,y groups for a specific experiment
#1.Generate expt specific X,y,groups
t1 = time.time()

X_expt , y_expt , groups_expt, expt_info = run_expts.generate_expt_x_y_groups(X,y,groups, 2 )
results_df = run_expts.run_mv_tsc(X_expt,y_expt,groups_expt, {'RF' : RandomForestClassifier(random_state = 5) , 'DT' : DecisionTreeClassifier() , 'Ada_B' : AdaBoostClassifier(random_state = 5)} , return_df = True , subset_names_and_cols = regional_features_dict, random_states = [1,2] )
res_df = run_expts.generate_subset_acc_std(results_df, return_df = True)

display(pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'RF' in col]].mean(axis = 0)).T)
display(pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'DT' in col]].mean(axis = 0)).T)
display(pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'Ada_B' in col]].mean(axis = 0)).T)


t2 = time.time()
time_taken = t2-t1
print(time_taken)

,Prefrontal_RF_acc,Frontal_RF_acc,Left Frontal_RF_acc,Right Frontal_RF_acc,Central_RF_acc,Left Central_RF_acc,Right Central_RF_acc,Left Temporal_RF_acc,Right Temporal_RF_acc,Parietal_RF_acc,Left Parietal_RF_acc,Right Parietal_RF_acc,Occipital_RF_acc
0,0.733929,0.501786,0.682143,0.55,0.248214,0.341071,0.342857,0.380357,0.482143,0.382143,0.444643,0.482143,0.483929


,Prefrontal_DT_acc,Frontal_DT_acc,Left Frontal_DT_acc,Right Frontal_DT_acc,Central_DT_acc,Left Central_DT_acc,Right Central_DT_acc,Left Temporal_DT_acc,Right Temporal_DT_acc,Parietal_DT_acc,Left Parietal_DT_acc,Right Parietal_DT_acc,Occipital_DT_acc
0,0.732143,0.4125,0.5375,0.5,0.316071,0.4125,0.378571,0.380357,0.555357,0.369643,0.485714,0.432143,0.523214


,Prefrontal_Ada_B_acc,Frontal_Ada_B_acc,Left Frontal_Ada_B_acc,Right Frontal_Ada_B_acc,Central_Ada_B_acc,Left Central_Ada_B_acc,Right Central_Ada_B_acc,Left Temporal_Ada_B_acc,Right Temporal_Ada_B_acc,Parietal_Ada_B_acc,Left Parietal_Ada_B_acc,Right Parietal_Ada_B_acc,Occipital_Ada_B_acc
0,0.651786,0.478571,0.646429,0.510714,0.369643,0.367857,0.483929,0.394643,0.457143,0.373214,0.396429,0.508929,0.519643


43.23227548599243


#### Combined Regions

In [59]:
#Now X, Y and groups have been loaded for N1 data - generate X,y groups for a specific experiment
#1.Generate expt specific X,y,groups
t1 = time.time()

X_expt , y_expt , groups_expt, expt_info = run_expts.generate_expt_x_y_groups(X,y,groups, 2)
results_df = run_expts.run_mv_tsc(X_expt,y_expt,groups_expt, {'RF' : RandomForestClassifier(random_state = 5) , 'DT' : DecisionTreeClassifier() , 'Ada_B' : AdaBoostClassifier(random_state = 5)} , return_df = True , subset_names_and_cols = combined_regions_features_dict, random_states = [1,2] )
res_df = run_expts.generate_subset_acc_std(results_df, return_df = True)

display(pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'RF' in col]].mean(axis = 0)).T)
display(pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'DT' in col]].mean(axis = 0)).T)
display(pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'Ada_B' in col]].mean(axis = 0)).T)


t2 = time.time()
time_taken = t2-t1
print(time_taken)

,Prefrontal_Frontal_RF_acc,Prefrontal_Left Frontal_RF_acc,Prefrontal_Right Frontal_RF_acc,Prefrontal_Central_RF_acc,Prefrontal_Left Central_RF_acc,Prefrontal_Right Central_RF_acc,Prefrontal_Left Temporal_RF_acc,Prefrontal_Right Temporal_RF_acc,Prefrontal_Parietal_RF_acc,Prefrontal_Left Parietal_RF_acc,Prefrontal_Right Parietal_RF_acc,Prefrontal_Occipital_RF_acc,Frontal_Left Frontal_RF_acc,Frontal_Right Frontal_RF_acc,Frontal_Central_RF_acc,Frontal_Left Central_RF_acc,Frontal_Right Central_RF_acc,Frontal_Left Temporal_RF_acc,Frontal_Right Temporal_RF_acc,Frontal_Parietal_RF_acc,Frontal_Left Parietal_RF_acc,Frontal_Right Parietal_RF_acc,Frontal_Occipital_RF_acc,Left Frontal_Right Frontal_RF_acc,Left Frontal_Central_RF_acc,Left Frontal_Left Central_RF_acc,Left Frontal_Right Central_RF_acc,Left Frontal_Left Temporal_RF_acc,Left Frontal_Right Temporal_RF_acc,Left Frontal_Parietal_RF_acc,Left Frontal_Left Parietal_RF_acc,Left Frontal_Right Parietal_RF_acc,Left Frontal_Occipital_RF_acc,Right Frontal_Central_RF_acc,Right Frontal_Left Central_RF_acc,Right Frontal_Right Central_RF_acc,Right Frontal_Left Temporal_RF_acc,Right Frontal_Right Temporal_RF_acc,Right Frontal_Parietal_RF_acc,Right Frontal_Left Parietal_RF_acc,Right Frontal_Right Parietal_RF_acc,Right Frontal_Occipital_RF_acc,Central_Left Central_RF_acc,Central_Right Central_RF_acc,Central_Left Temporal_RF_acc,Central_Right Temporal_RF_acc,Central_Parietal_RF_acc,Central_Left Parietal_RF_acc,Central_Right Parietal_RF_acc,Central_Occipital_RF_acc,Left Central_Right Central_RF_acc,Left Central_Left Temporal_RF_acc,Left Central_Right Temporal_RF_acc,Left Central_Parietal_RF_acc,Left Central_Left Parietal_RF_acc,Left Central_Right Parietal_RF_acc,Left Central_Occipital_RF_acc,Right Central_Left Temporal_RF_acc,Right Central_Right Temporal_RF_acc,Right Central_Parietal_RF_acc,Right Central_Left Parietal_RF_acc,Right Central_Right Parietal_RF_acc,Right Central_Occipital_RF_acc,Left Temporal_Right Temporal_RF_acc,Left Temporal_Parietal_RF_acc,Left Temporal_Left Parietal_RF_acc,Left Temporal_Right Parietal_RF_acc,Left Temporal_Occipital_RF_acc,Right Temporal_Parietal_RF_acc,Right Temporal_Left Parietal_RF_acc,Right Temporal_Right Parietal_RF_acc,Right Temporal_Occipital_RF_acc,Parietal_Left Parietal_RF_acc,Parietal_Right Parietal_RF_acc,Parietal_Occipital_RF_acc,Left Parietal_Right Parietal_RF_acc,Left Parietal_Occipital_RF_acc,Right Parietal_Occipital_RF_acc
0,0.680357,0.680357,0.6375,0.680357,0.735714,0.669643,0.708929,0.680357,0.748214,0.694643,0.733929,0.719643,0.619643,0.485714,0.358929,0.378571,0.367857,0.382143,0.514286,0.405357,0.539286,0.4625,0.432143,0.558929,0.576786,0.578571,0.567857,0.657143,0.660714,0.667857,0.589286,0.644643,0.694643,0.475,0.496429,0.473214,0.548214,0.5375,0.469643,0.548214,0.5375,0.5875,0.275,0.3125,0.382143,0.380357,0.317857,0.394643,0.341071,0.285714,0.3125,0.332143,0.367857,0.289286,0.392857,0.394643,0.344643,0.292857,0.442857,0.328571,0.407143,0.341071,0.364286,0.501786,0.330357,0.328571,0.378571,0.425,0.482143,0.5375,0.405357,0.485714,0.364286,0.342857,0.367857,0.416071,0.433929,0.416071


,Prefrontal_Frontal_DT_acc,Prefrontal_Left Frontal_DT_acc,Prefrontal_Right Frontal_DT_acc,Prefrontal_Central_DT_acc,Prefrontal_Left Central_DT_acc,Prefrontal_Right Central_DT_acc,Prefrontal_Left Temporal_DT_acc,Prefrontal_Right Temporal_DT_acc,Prefrontal_Parietal_DT_acc,Prefrontal_Left Parietal_DT_acc,Prefrontal_Right Parietal_DT_acc,Prefrontal_Occipital_DT_acc,Frontal_Left Frontal_DT_acc,Frontal_Right Frontal_DT_acc,Frontal_Central_DT_acc,Frontal_Left Central_DT_acc,Frontal_Right Central_DT_acc,Frontal_Left Temporal_DT_acc,Frontal_Right Temporal_DT_acc,Frontal_Parietal_DT_acc,Frontal_Left Parietal_DT_acc,Frontal_Right Parietal_DT_acc,Frontal_Occipital_DT_acc,Left Frontal_Right Frontal_DT_acc,Left Frontal_Central_DT_acc,Left Frontal_Left Central_DT_acc,Left Frontal_Right Central_DT_acc,Left Frontal_Left Temporal_DT_acc,Left Frontal_Right Temporal_DT_acc,Left Frontal_Parietal_DT_acc,Left Frontal_Left Parietal_DT_acc,Left Frontal_Right Parietal_DT_acc,Left Frontal_Occipital_DT_acc,Right Frontal_Central_DT_acc,Right Frontal_Left Central_DT_acc,Right Frontal_Right Central_DT_acc,Right Frontal_Left Temporal_DT_acc,Right Frontal_Right Temporal_DT_acc,Right Frontal_Parietal_DT_acc,Right Frontal_Left Parietal_DT_acc,Right Frontal_Right Parietal_DT_acc,Right Frontal_Occipital_DT_acc,Central_Left Central_DT_acc,Central_Right Central_DT_acc,Central_Left Temporal_DT_acc,Central_Right Temporal_DT_acc,Central_Parietal_DT_acc,Central_Left Parietal_DT_acc,Central_Right Parietal_DT_acc,Central_Occipital_DT_acc,Left Central_Right Central_DT_acc,Left Central_Left Temporal_DT_acc,Left Central_Right Temporal_DT_acc,Left Central_Parietal_DT_acc,Left Central_Left Parietal_DT_acc,Left Central_Right Parietal_DT_acc,Left Central_Occipital_DT_acc,Right Central_Left Temporal_DT_acc,Right Central_Right Temporal_DT_acc,Right Central_Parietal_DT_acc,Right Central_Left Parietal_DT_acc,Right Central_Right Parietal_DT_acc,Right Central_Occipital_DT_acc,Left Temporal_Right Temporal_DT_acc,Left Temporal_Parietal_DT_acc,Left Temporal_Left Parietal_DT_acc,Left Temporal_Right Parietal_DT_acc,Left Temporal_Occipital_DT_acc,Right Temporal_Parietal_DT_acc,Right Temporal_Left Parietal_DT_acc,Right Temporal_Right Parietal_DT_acc,Right Temporal_Occipital_DT_acc,Parietal_Left Parietal_DT_acc,Parietal_Right Parietal_DT_acc,Parietal_Occipital_DT_acc,Left Parietal_Right Parietal_DT_acc,Left Parietal_Occipital_DT_acc,Right Parietal_Occipital_DT_acc
0,0.707143,0.657143,0.635714,0.773214,0.678571,0.703571,0.855357,0.733929,0.707143,0.717857,0.758929,0.692857,0.6,0.446429,0.432143,0.501786,0.383929,0.4125,0.514286,0.335714,0.433929,0.416071,0.417857,0.471429,0.560714,0.458929,0.516071,0.585714,0.701786,0.564286,0.607143,0.566071,0.560714,0.4375,0.460714,0.489286,0.501786,0.580357,0.539286,0.553571,0.526786,0.605357,0.264286,0.291071,0.357143,0.45,0.25,0.375,0.326786,0.380357,0.3,0.371429,0.451786,0.223214,0.367857,0.389286,0.392857,0.316071,0.5125,0.369643,0.421429,0.401786,0.351786,0.592857,0.328571,0.394643,0.3,0.453571,0.460714,0.505357,0.521429,0.526786,0.383929,0.351786,0.435714,0.339286,0.369643,0.417857


,Prefrontal_Frontal_Ada_B_acc,Prefrontal_Left Frontal_Ada_B_acc,Prefrontal_Right Frontal_Ada_B_acc,Prefrontal_Central_Ada_B_acc,Prefrontal_Left Central_Ada_B_acc,Prefrontal_Right Central_Ada_B_acc,Prefrontal_Left Temporal_Ada_B_acc,Prefrontal_Right Temporal_Ada_B_acc,Prefrontal_Parietal_Ada_B_acc,Prefrontal_Left Parietal_Ada_B_acc,Prefrontal_Right Parietal_Ada_B_acc,Prefrontal_Occipital_Ada_B_acc,Frontal_Left Frontal_Ada_B_acc,Frontal_Right Frontal_Ada_B_acc,Frontal_Central_Ada_B_acc,Frontal_Left Central_Ada_B_acc,Frontal_Right Central_Ada_B_acc,Frontal_Left Temporal_Ada_B_acc,Frontal_Right Temporal_Ada_B_acc,Frontal_Parietal_Ada_B_acc,Frontal_Left Parietal_Ada_B_acc,Frontal_Right Parietal_Ada_B_acc,Frontal_Occipital_Ada_B_acc,Left Frontal_Right Frontal_Ada_B_acc,Left Frontal_Central_Ada_B_acc,Left Frontal_Left Central_Ada_B_acc,Left Frontal_Right Central_Ada_B_acc,Left Frontal_Left Temporal_Ada_B_acc,Left Frontal_Right Temporal_Ada_B_acc,Left Frontal_Parietal_Ada_B_acc,Left Frontal_Left Parietal_Ada_B_acc,Left Frontal_Right Parietal_Ada_B_acc,Left Frontal_Occipital_Ada_B_acc,Right Frontal_Central_Ada_B_acc,Right Frontal_Left Central_Ada_B_acc,Right Frontal_Right Central_Ada_B_acc,Right Frontal_Left Temporal_Ada_B_acc,Right Frontal_Right Temporal_Ada_B_acc,Right Frontal_Parietal_Ada_B_acc,Right Frontal_Left Parietal_Ada_B_acc,Right Frontal_Right Parietal_Ada_B_acc,Right Frontal_Occipital_Ada_B_acc,Central_Left Central_Ada_B_acc,Central_Right Central_Ada_B_acc,Central_Left Temporal_Ada_B_acc,Central_Right Temporal_Ada_B_acc,Central_Parietal_Ada_B_acc,Central_Left Parietal_Ada_B_acc,Central_Right Parietal_Ada_B_acc,Central_Occipital_Ada_B_acc,Left Central_Right Central_Ada_B_acc,Left Central_Left Temporal_Ada_B_acc,Left Central_Right Temporal_Ada_B_acc,Left Central_Parietal_Ada_B_acc,Left Central_Left Parietal_Ada_B_acc,Left Central_Right Parietal_Ada_B_acc,Left Central_Occipital_Ada_B_acc,Right Central_Left Temporal_Ada_B_acc,Right Central_Right Temporal_Ada_B_acc,Right Central_Parietal_Ada_B_acc,Right Central_Left Parietal_Ada_B_acc,Right Central_Right Parietal_Ada_B_acc,Right Central_Occipital_Ada_B_acc,Left Temporal_Right Temporal_Ada_B_acc,Left Temporal_Parietal_Ada_B_acc,Left Temporal_Left Parietal_Ada_B_acc,Left Temporal_Right Parietal_Ada_B_acc,Left Temporal_Occipital_Ada_B_acc,Right Temporal_Parietal_Ada_B_acc,Right Temporal_Left Parietal_Ada_B_acc,Right Temporal_Right Parietal_Ada_B_acc,Right Temporal_Occipital_Ada_B_acc,Parietal_Left Parietal_Ada_B_acc,Parietal_Right Parietal_Ada_B_acc,Parietal_Occipital_Ada_B_acc,Left Parietal_Right Parietal_Ada_B_acc,Left Parietal_Occipital_Ada_B_acc,Right Parietal_Occipital_Ada_B_acc
0,0.65,0.667857,0.667857,0.630357,0.707143,0.694643,0.869643,0.667857,0.707143,0.667857,0.630357,0.641071,0.580357,0.473214,0.392857,0.394643,0.378571,0.442857,0.548214,0.391071,0.435714,0.396429,0.483929,0.526786,0.55,0.535714,0.546429,0.633929,0.575,0.539286,0.523214,0.576786,0.617857,0.442857,0.457143,0.5,0.525,0.5125,0.482143,0.482143,0.564286,0.553571,0.353571,0.460714,0.385714,0.391071,0.291071,0.328571,0.408929,0.398214,0.403571,0.291071,0.475,0.392857,0.455357,0.380357,0.460714,0.417857,0.496429,0.366071,0.535714,0.428571,0.428571,0.501786,0.355357,0.316071,0.426786,0.489286,0.380357,0.551786,0.442857,0.471429,0.310714,0.403571,0.369643,0.521429,0.416071,0.457143


115.62148976325989


In [66]:
res_df[[col for col in res_df.columns if 'Prefrontal' in col and 'Ada_B' in col]]

,Prefrontal_Frontal_Ada_B_acc,Prefrontal_Frontal_Ada_B_std,Prefrontal_Left Frontal_Ada_B_acc,Prefrontal_Left Frontal_Ada_B_std,Prefrontal_Right Frontal_Ada_B_acc,Prefrontal_Right Frontal_Ada_B_std,Prefrontal_Central_Ada_B_acc,Prefrontal_Central_Ada_B_std,Prefrontal_Left Central_Ada_B_acc,Prefrontal_Left Central_Ada_B_std,Prefrontal_Right Central_Ada_B_acc,Prefrontal_Right Central_Ada_B_std,Prefrontal_Left Temporal_Ada_B_acc,Prefrontal_Left Temporal_Ada_B_std,Prefrontal_Right Temporal_Ada_B_acc,Prefrontal_Right Temporal_Ada_B_std,Prefrontal_Parietal_Ada_B_acc,Prefrontal_Parietal_Ada_B_std,Prefrontal_Left Parietal_Ada_B_acc,Prefrontal_Left Parietal_Ada_B_std,Prefrontal_Right Parietal_Ada_B_acc,Prefrontal_Right Parietal_Ada_B_std,Prefrontal_Occipital_Ada_B_acc,Prefrontal_Occipital_Ada_B_std
0,0.735714,0.017496,0.653571,0.121429,0.714286,0.117369,0.553571,0.129264,0.710714,0.045737,0.603571,0.166139,0.896429,0.094761,0.657143,0.065270,0.657143,0.065270,0.628571,0.111461,0.603571,0.12289,0.632143,0.091473
1,0.564286,0.227453,0.682143,0.146559,0.621429,0.181547,0.707143,0.069253,0.703571,0.147945,0.785714,0.116824,0.842857,0.047110,0.678571,0.087482,0.757143,0.113277,0.707143,0.069253,0.657143,0.06527,0.650000,0.152613


In [64]:
display(pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'RF' in col]].mean(axis = 0)).max())
display(pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'DT' in col]].mean(axis = 0)).max())
display(pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'Ada_B' in col]].mean(axis = 0)).max())

0    0.748214
dtype: float64

0    0.855357
dtype: float64

0    0.869643
dtype: float64

In [63]:
pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'RF' in col]].mean(axis = 0)).max()

0    0.748214
dtype: float64

### Using all of the data 

In [46]:
#Now X, Y and groups have been loaded for N1 data - generate X,y groups for a specific experiment
#1.Generate expt specific X,y,groups
t1 = time.time()

X_expt , y_expt , groups_expt, expt_info = run_expts.generate_expt_x_y_groups(X,y,groups, 2 )
results_df = run_expts.run_mv_tsc(X_expt,y_expt,groups_expt, {'RF' : RandomForestClassifier(random_state = 5) , 'DT' : DecisionTreeClassifier() , 'Ada_B' : AdaBoostClassifier(random_state = 5)} , return_df = True , subset_names_and_cols = all_data_dict, random_states = [1,2] )
res_df = run_expts.generate_subset_acc_std(results_df, return_df = True)

display(pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'RF' in col]].mean(axis = 0)).T)
display(pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'DT' in col]].mean(axis = 0)).T)
display(pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'Ada_B' in col]].mean(axis = 0)).T)


t2 = time.time()
time_taken = t2-t1
print(time_taken)

,All_regions_RF_acc,All_regions_2_RF_acc
0,0.666071,0.666071


,All_regions_DT_acc,All_regions_2_DT_acc
0,0.758929,0.7


,All_regions_Ada_B_acc,All_regions_2_Ada_B_acc
0,0.746429,0.746429


3.6197476387023926


## Expt 3

### Regional

In [28]:
#Now X, Y and groups have been loaded for N1 data - generate X,y groups for a specific experiment
#1.Generate expt specific X,y,groups
t1 = time.time()

X_expt , y_expt , groups_expt, expt_info = run_expts.generate_expt_x_y_groups(X,y,groups, 3 )
results_df = run_expts.run_mv_tsc(X_expt,y_expt,groups_expt, {'RF' : RandomForestClassifier(random_state = 5) , 'DT' : DecisionTreeClassifier() , 'Ada_B' : AdaBoostClassifier(random_state = 5)} , return_df = True , subset_names_and_cols = regional_features_dict, random_states = [1,2] )
res_df = run_expts.generate_subset_acc_std(results_df, return_df = True)

display(pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'RF' in col]].mean(axis = 0)).T)
display(pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'DT' in col]].mean(axis = 0)).T)
display(pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'Ada_B' in col]].mean(axis = 0)).T)


t2 = time.time()
time_taken = t2-t1
print(time_taken)

,Prefrontal_RF_acc,Frontal_RF_acc,Left Frontal_RF_acc,Right Frontal_RF_acc,Central_RF_acc,Left Central_RF_acc,Right Central_RF_acc,Left Temporal_RF_acc,Right Temporal_RF_acc,Parietal_RF_acc,Left Parietal_RF_acc,Right Parietal_RF_acc,Occipital_RF_acc
0,0.590152,0.284091,0.393182,0.375,0.160606,0.168182,0.256818,0.187121,0.266667,0.231818,0.256818,0.265152,0.337879


,Prefrontal_DT_acc,Frontal_DT_acc,Left Frontal_DT_acc,Right Frontal_DT_acc,Central_DT_acc,Left Central_DT_acc,Right Central_DT_acc,Left Temporal_DT_acc,Right Temporal_DT_acc,Parietal_DT_acc,Left Parietal_DT_acc,Right Parietal_DT_acc,Occipital_DT_acc
0,0.545455,0.293939,0.375758,0.30303,0.215152,0.248485,0.347727,0.25,0.349242,0.302273,0.212879,0.336364,0.339394


,Prefrontal_Ada_B_acc,Frontal_Ada_B_acc,Left Frontal_Ada_B_acc,Right Frontal_Ada_B_acc,Central_Ada_B_acc,Left Central_Ada_B_acc,Right Central_Ada_B_acc,Left Temporal_Ada_B_acc,Right Temporal_Ada_B_acc,Parietal_Ada_B_acc,Left Parietal_Ada_B_acc,Right Parietal_Ada_B_acc,Occipital_Ada_B_acc
0,0.563636,0.256818,0.329545,0.286364,0.312879,0.284091,0.328788,0.287879,0.259091,0.259848,0.239394,0.319697,0.277273


33.791120529174805


### Combined Regions

In [67]:
#Now X, Y and groups have been loaded for N1 data - generate X,y groups for a specific experiment
#1.Generate expt specific X,y,groups
t1 = time.time()

X_expt , y_expt , groups_expt, expt_info = run_expts.generate_expt_x_y_groups(X,y,groups, 3)
results_df = run_expts.run_mv_tsc(X_expt,y_expt,groups_expt, {'RF' : RandomForestClassifier(random_state = 5) , 'DT' : DecisionTreeClassifier() , 'Ada_B' : AdaBoostClassifier(random_state = 5)} , return_df = True , subset_names_and_cols = combined_regions_features_dict, random_states = [1,2] )
res_df = run_expts.generate_subset_acc_std(results_df, return_df = True)

display(pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'RF' in col]].mean(axis = 0)).T)
display(pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'DT' in col]].mean(axis = 0)).T)
display(pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'Ada_B' in col]].mean(axis = 0)).T)


t2 = time.time()
time_taken = t2-t1
print(time_taken)

,Prefrontal_Frontal_RF_acc,Prefrontal_Left Frontal_RF_acc,Prefrontal_Right Frontal_RF_acc,Prefrontal_Central_RF_acc,Prefrontal_Left Central_RF_acc,Prefrontal_Right Central_RF_acc,Prefrontal_Left Temporal_RF_acc,Prefrontal_Right Temporal_RF_acc,Prefrontal_Parietal_RF_acc,Prefrontal_Left Parietal_RF_acc,Prefrontal_Right Parietal_RF_acc,Prefrontal_Occipital_RF_acc,Frontal_Left Frontal_RF_acc,Frontal_Right Frontal_RF_acc,Frontal_Central_RF_acc,Frontal_Left Central_RF_acc,Frontal_Right Central_RF_acc,Frontal_Left Temporal_RF_acc,Frontal_Right Temporal_RF_acc,Frontal_Parietal_RF_acc,Frontal_Left Parietal_RF_acc,Frontal_Right Parietal_RF_acc,Frontal_Occipital_RF_acc,Left Frontal_Right Frontal_RF_acc,Left Frontal_Central_RF_acc,Left Frontal_Left Central_RF_acc,Left Frontal_Right Central_RF_acc,Left Frontal_Left Temporal_RF_acc,Left Frontal_Right Temporal_RF_acc,Left Frontal_Parietal_RF_acc,Left Frontal_Left Parietal_RF_acc,Left Frontal_Right Parietal_RF_acc,Left Frontal_Occipital_RF_acc,Right Frontal_Central_RF_acc,Right Frontal_Left Central_RF_acc,Right Frontal_Right Central_RF_acc,Right Frontal_Left Temporal_RF_acc,Right Frontal_Right Temporal_RF_acc,Right Frontal_Parietal_RF_acc,Right Frontal_Left Parietal_RF_acc,Right Frontal_Right Parietal_RF_acc,Right Frontal_Occipital_RF_acc,Central_Left Central_RF_acc,Central_Right Central_RF_acc,Central_Left Temporal_RF_acc,Central_Right Temporal_RF_acc,Central_Parietal_RF_acc,Central_Left Parietal_RF_acc,Central_Right Parietal_RF_acc,Central_Occipital_RF_acc,Left Central_Right Central_RF_acc,Left Central_Left Temporal_RF_acc,Left Central_Right Temporal_RF_acc,Left Central_Parietal_RF_acc,Left Central_Left Parietal_RF_acc,Left Central_Right Parietal_RF_acc,Left Central_Occipital_RF_acc,Right Central_Left Temporal_RF_acc,Right Central_Right Temporal_RF_acc,Right Central_Parietal_RF_acc,Right Central_Left Parietal_RF_acc,Right Central_Right Parietal_RF_acc,Right Central_Occipital_RF_acc,Left Temporal_Right Temporal_RF_acc,Left Temporal_Parietal_RF_acc,Left Temporal_Left Parietal_RF_acc,Left Temporal_Right Parietal_RF_acc,Left Temporal_Occipital_RF_acc,Right Temporal_Parietal_RF_acc,Right Temporal_Left Parietal_RF_acc,Right Temporal_Right Parietal_RF_acc,Right Temporal_Occipital_RF_acc,Parietal_Left Parietal_RF_acc,Parietal_Right Parietal_RF_acc,Parietal_Occipital_RF_acc,Left Parietal_Right Parietal_RF_acc,Left Parietal_Occipital_RF_acc,Right Parietal_Occipital_RF_acc
0,0.517424,0.529545,0.473485,0.437879,0.446212,0.472727,0.500758,0.518182,0.520455,0.509091,0.561364,0.554545,0.400758,0.365152,0.267424,0.211364,0.300758,0.220455,0.267424,0.266667,0.24697,0.319697,0.219697,0.382576,0.340152,0.373485,0.34697,0.358333,0.420455,0.465909,0.418939,0.453788,0.410606,0.285606,0.294697,0.32803,0.321212,0.375758,0.311364,0.329545,0.382576,0.314394,0.12197,0.240909,0.159091,0.196212,0.249242,0.193182,0.159848,0.115152,0.185606,0.151515,0.204545,0.195455,0.175758,0.20303,0.115152,0.230303,0.321212,0.231818,0.274242,0.230303,0.177273,0.265909,0.25,0.214394,0.258333,0.187121,0.30303,0.321212,0.284848,0.274242,0.215152,0.223485,0.204545,0.255303,0.264394,0.22803


,Prefrontal_Frontal_DT_acc,Prefrontal_Left Frontal_DT_acc,Prefrontal_Right Frontal_DT_acc,Prefrontal_Central_DT_acc,Prefrontal_Left Central_DT_acc,Prefrontal_Right Central_DT_acc,Prefrontal_Left Temporal_DT_acc,Prefrontal_Right Temporal_DT_acc,Prefrontal_Parietal_DT_acc,Prefrontal_Left Parietal_DT_acc,Prefrontal_Right Parietal_DT_acc,Prefrontal_Occipital_DT_acc,Frontal_Left Frontal_DT_acc,Frontal_Right Frontal_DT_acc,Frontal_Central_DT_acc,Frontal_Left Central_DT_acc,Frontal_Right Central_DT_acc,Frontal_Left Temporal_DT_acc,Frontal_Right Temporal_DT_acc,Frontal_Parietal_DT_acc,Frontal_Left Parietal_DT_acc,Frontal_Right Parietal_DT_acc,Frontal_Occipital_DT_acc,Left Frontal_Right Frontal_DT_acc,Left Frontal_Central_DT_acc,Left Frontal_Left Central_DT_acc,Left Frontal_Right Central_DT_acc,Left Frontal_Left Temporal_DT_acc,Left Frontal_Right Temporal_DT_acc,Left Frontal_Parietal_DT_acc,Left Frontal_Left Parietal_DT_acc,Left Frontal_Right Parietal_DT_acc,Left Frontal_Occipital_DT_acc,Right Frontal_Central_DT_acc,Right Frontal_Left Central_DT_acc,Right Frontal_Right Central_DT_acc,Right Frontal_Left Temporal_DT_acc,Right Frontal_Right Temporal_DT_acc,Right Frontal_Parietal_DT_acc,Right Frontal_Left Parietal_DT_acc,Right Frontal_Right Parietal_DT_acc,Right Frontal_Occipital_DT_acc,Central_Left Central_DT_acc,Central_Right Central_DT_acc,Central_Left Temporal_DT_acc,Central_Right Temporal_DT_acc,Central_Parietal_DT_acc,Central_Left Parietal_DT_acc,Central_Right Parietal_DT_acc,Central_Occipital_DT_acc,Left Central_Right Central_DT_acc,Left Central_Left Temporal_DT_acc,Left Central_Right Temporal_DT_acc,Left Central_Parietal_DT_acc,Left Central_Left Parietal_DT_acc,Left Central_Right Parietal_DT_acc,Left Central_Occipital_DT_acc,Right Central_Left Temporal_DT_acc,Right Central_Right Temporal_DT_acc,Right Central_Parietal_DT_acc,Right Central_Left Parietal_DT_acc,Right Central_Right Parietal_DT_acc,Right Central_Occipital_DT_acc,Left Temporal_Right Temporal_DT_acc,Left Temporal_Parietal_DT_acc,Left Temporal_Left Parietal_DT_acc,Left Temporal_Right Parietal_DT_acc,Left Temporal_Occipital_DT_acc,Right Temporal_Parietal_DT_acc,Right Temporal_Left Parietal_DT_acc,Right Temporal_Right Parietal_DT_acc,Right Temporal_Occipital_DT_acc,Parietal_Left Parietal_DT_acc,Parietal_Right Parietal_DT_acc,Parietal_Occipital_DT_acc,Left Parietal_Right Parietal_DT_acc,Left Parietal_Occipital_DT_acc,Right Parietal_Occipital_DT_acc
0,0.403788,0.465152,0.427273,0.409848,0.465152,0.465152,0.563636,0.443939,0.492424,0.403788,0.447727,0.446212,0.329545,0.303788,0.241667,0.310606,0.259848,0.292424,0.322727,0.312121,0.240152,0.310606,0.284848,0.312879,0.375,0.373485,0.340152,0.392424,0.419697,0.349242,0.319697,0.40303,0.471212,0.302273,0.277273,0.331818,0.347727,0.381818,0.313636,0.294697,0.339394,0.37803,0.251515,0.32197,0.213636,0.275758,0.224242,0.205303,0.313636,0.223485,0.266667,0.258333,0.313636,0.248485,0.296212,0.312879,0.284091,0.242424,0.356061,0.225,0.187121,0.321212,0.22197,0.312879,0.329545,0.242424,0.266667,0.185606,0.401515,0.258333,0.304545,0.305303,0.222727,0.302273,0.240909,0.274242,0.212879,0.248485


,Prefrontal_Frontal_Ada_B_acc,Prefrontal_Left Frontal_Ada_B_acc,Prefrontal_Right Frontal_Ada_B_acc,Prefrontal_Central_Ada_B_acc,Prefrontal_Left Central_Ada_B_acc,Prefrontal_Right Central_Ada_B_acc,Prefrontal_Left Temporal_Ada_B_acc,Prefrontal_Right Temporal_Ada_B_acc,Prefrontal_Parietal_Ada_B_acc,Prefrontal_Left Parietal_Ada_B_acc,Prefrontal_Right Parietal_Ada_B_acc,Prefrontal_Occipital_Ada_B_acc,Frontal_Left Frontal_Ada_B_acc,Frontal_Right Frontal_Ada_B_acc,Frontal_Central_Ada_B_acc,Frontal_Left Central_Ada_B_acc,Frontal_Right Central_Ada_B_acc,Frontal_Left Temporal_Ada_B_acc,Frontal_Right Temporal_Ada_B_acc,Frontal_Parietal_Ada_B_acc,Frontal_Left Parietal_Ada_B_acc,Frontal_Right Parietal_Ada_B_acc,Frontal_Occipital_Ada_B_acc,Left Frontal_Right Frontal_Ada_B_acc,Left Frontal_Central_Ada_B_acc,Left Frontal_Left Central_Ada_B_acc,Left Frontal_Right Central_Ada_B_acc,Left Frontal_Left Temporal_Ada_B_acc,Left Frontal_Right Temporal_Ada_B_acc,Left Frontal_Parietal_Ada_B_acc,Left Frontal_Left Parietal_Ada_B_acc,Left Frontal_Right Parietal_Ada_B_acc,Left Frontal_Occipital_Ada_B_acc,Right Frontal_Central_Ada_B_acc,Right Frontal_Left Central_Ada_B_acc,Right Frontal_Right Central_Ada_B_acc,Right Frontal_Left Temporal_Ada_B_acc,Right Frontal_Right Temporal_Ada_B_acc,Right Frontal_Parietal_Ada_B_acc,Right Frontal_Left Parietal_Ada_B_acc,Right Frontal_Right Parietal_Ada_B_acc,Right Frontal_Occipital_Ada_B_acc,Central_Left Central_Ada_B_acc,Central_Right Central_Ada_B_acc,Central_Left Temporal_Ada_B_acc,Central_Right Temporal_Ada_B_acc,Central_Parietal_Ada_B_acc,Central_Left Parietal_Ada_B_acc,Central_Right Parietal_Ada_B_acc,Central_Occipital_Ada_B_acc,Left Central_Right Central_Ada_B_acc,Left Central_Left Temporal_Ada_B_acc,Left Central_Right Temporal_Ada_B_acc,Left Central_Parietal_Ada_B_acc,Left Central_Left Parietal_Ada_B_acc,Left Central_Right Parietal_Ada_B_acc,Left Central_Occipital_Ada_B_acc,Right Central_Left Temporal_Ada_B_acc,Right Central_Right Temporal_Ada_B_acc,Right Central_Parietal_Ada_B_acc,Right Central_Left Parietal_Ada_B_acc,Right Central_Right Parietal_Ada_B_acc,Right Central_Occipital_Ada_B_acc,Left Temporal_Right Temporal_Ada_B_acc,Left Temporal_Parietal_Ada_B_acc,Left Temporal_Left Parietal_Ada_B_acc,Left Temporal_Right Parietal_Ada_B_acc,Left Temporal_Occipital_Ada_B_acc,Right Temporal_Parietal_Ada_B_acc,Right Temporal_Left Parietal_Ada_B_acc,Right Temporal_Right Parietal_Ada_B_acc,Right Temporal_Occipital_Ada_B_acc,Parietal_Left Parietal_Ada_B_acc,Parietal_Right Parietal_Ada_B_acc,Parietal_Occipital_Ada_B_acc,Left Parietal_Right Parietal_Ada_B_acc,Left Parietal_Occipital_Ada_B_acc,Right Parietal_Occipital_Ada_B_acc
0,0.498485,0.490152,0.490909,0.403788,0.507576,0.482576,0.536364,0.456061,0.500758,0.439394,0.375758,0.42197,0.302273,0.374242,0.322727,0.250758,0.276515,0.223485,0.258333,0.27803,0.266667,0.292424,0.239394,0.357576,0.330303,0.331061,0.328788,0.356061,0.384848,0.330303,0.367424,0.321212,0.374242,0.366667,0.337879,0.32803,0.330303,0.339394,0.312121,0.349242,0.331061,0.288636,0.213636,0.32197,0.238636,0.294697,0.329545,0.249242,0.305303,0.287879,0.309848,0.240909,0.240909,0.286364,0.259091,0.277273,0.231061,0.276515,0.284848,0.259848,0.312879,0.301515,0.320455,0.322727,0.337879,0.22197,0.318939,0.268182,0.286364,0.321212,0.231818,0.276515,0.267424,0.319697,0.212121,0.339394,0.255303,0.266667


119.92189621925354


In [68]:
display(pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'RF' in col]].mean(axis = 0)).max())
display(pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'DT' in col]].mean(axis = 0)).max())
display(pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'Ada_B' in col]].mean(axis = 0)).max())

0    0.561364
dtype: float64

0    0.563636
dtype: float64

0    0.536364
dtype: float64

### Using all of the data 

In [47]:
#Now X, Y and groups have been loaded for N1 data - generate X,y groups for a specific experiment
#1.Generate expt specific X,y,groups
t1 = time.time()

X_expt , y_expt , groups_expt, expt_info = run_expts.generate_expt_x_y_groups(X,y,groups, 3 )
results_df = run_expts.run_mv_tsc(X_expt,y_expt,groups_expt, {'RF' : RandomForestClassifier(random_state = 5) , 'DT' : DecisionTreeClassifier() , 'Ada_B' : AdaBoostClassifier(random_state = 5)} , return_df = True , subset_names_and_cols = all_data_dict, random_states = [1,2] )
res_df = run_expts.generate_subset_acc_std(results_df, return_df = True)

display(pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'RF' in col]].mean(axis = 0)).T)
display(pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'DT' in col]].mean(axis = 0)).T)
display(pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'Ada_B' in col]].mean(axis = 0)).T)


t2 = time.time()
time_taken = t2-t1
print(time_taken)

,All_regions_RF_acc,All_regions_2_RF_acc
0,0.426515,0.426515


,All_regions_DT_acc,All_regions_2_DT_acc
0,0.455303,0.457576


,All_regions_Ada_B_acc,All_regions_2_Ada_B_acc
0,0.437121,0.437121


3.9537250995635986


## Expt 4

### Regional

In [29]:
#Now X, Y and groups have been loaded for N1 data - generate X,y groups for a specific experiment
#1.Generate expt specific X,y,groups
t1 = time.time()

X_expt , y_expt , groups_expt, expt_info = run_expts.generate_expt_x_y_groups(X,y,groups, 4 )
results_df = run_expts.run_mv_tsc(X_expt,y_expt,groups_expt, {'RF' : RandomForestClassifier(random_state = 5) , 'DT' : DecisionTreeClassifier() , 'Ada_B' : AdaBoostClassifier(random_state = 5)} , return_df = True , subset_names_and_cols = regional_features_dict, random_states = [1,2] )
res_df = run_expts.generate_subset_acc_std(results_df, return_df = True)

display(pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'RF' in col]].mean(axis = 0)).T)
display(pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'DT' in col]].mean(axis = 0)).T)
display(pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'Ada_B' in col]].mean(axis = 0)).T)


t2 = time.time()
time_taken = t2-t1
print(time_taken)

,Prefrontal_RF_acc,Frontal_RF_acc,Left Frontal_RF_acc,Right Frontal_RF_acc,Central_RF_acc,Left Central_RF_acc,Right Central_RF_acc,Left Temporal_RF_acc,Right Temporal_RF_acc,Parietal_RF_acc,Left Parietal_RF_acc,Right Parietal_RF_acc,Occipital_RF_acc
0,0.420455,0.275,0.294697,0.268182,0.160606,0.205303,0.258333,0.223485,0.213636,0.233333,0.237879,0.193182,0.230303


,Prefrontal_DT_acc,Frontal_DT_acc,Left Frontal_DT_acc,Right Frontal_DT_acc,Central_DT_acc,Left Central_DT_acc,Right Central_DT_acc,Left Temporal_DT_acc,Right Temporal_DT_acc,Parietal_DT_acc,Left Parietal_DT_acc,Right Parietal_DT_acc,Occipital_DT_acc
0,0.384091,0.330303,0.215152,0.275,0.279545,0.238636,0.249242,0.27803,0.331818,0.286364,0.256061,0.257576,0.258333


,Prefrontal_Ada_B_acc,Frontal_Ada_B_acc,Left Frontal_Ada_B_acc,Right Frontal_Ada_B_acc,Central_Ada_B_acc,Left Central_Ada_B_acc,Right Central_Ada_B_acc,Left Temporal_Ada_B_acc,Right Temporal_Ada_B_acc,Parietal_Ada_B_acc,Left Parietal_Ada_B_acc,Right Parietal_Ada_B_acc,Occipital_Ada_B_acc
0,0.354545,0.258333,0.310606,0.277273,0.250758,0.193182,0.215152,0.357576,0.248485,0.240152,0.194697,0.260606,0.17803


33.829728841781616


### Combined Regions

In [69]:
#Now X, Y and groups have been loaded for N1 data - generate X,y groups for a specific experiment
#1.Generate expt specific X,y,groups
t1 = time.time()

X_expt , y_expt , groups_expt, expt_info = run_expts.generate_expt_x_y_groups(X,y,groups, 4)
results_df = run_expts.run_mv_tsc(X_expt,y_expt,groups_expt, {'RF' : RandomForestClassifier(random_state = 5) , 'DT' : DecisionTreeClassifier() , 'Ada_B' : AdaBoostClassifier(random_state = 5)} , return_df = True , subset_names_and_cols = combined_regions_features_dict, random_states = [1,2] )
res_df = run_expts.generate_subset_acc_std(results_df, return_df = True)

display(pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'RF' in col]].mean(axis = 0)).T)
display(pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'DT' in col]].mean(axis = 0)).T)
display(pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'Ada_B' in col]].mean(axis = 0)).T)


t2 = time.time()
time_taken = t2-t1
print(time_taken)

,Prefrontal_Frontal_RF_acc,Prefrontal_Left Frontal_RF_acc,Prefrontal_Right Frontal_RF_acc,Prefrontal_Central_RF_acc,Prefrontal_Left Central_RF_acc,Prefrontal_Right Central_RF_acc,Prefrontal_Left Temporal_RF_acc,Prefrontal_Right Temporal_RF_acc,Prefrontal_Parietal_RF_acc,Prefrontal_Left Parietal_RF_acc,Prefrontal_Right Parietal_RF_acc,Prefrontal_Occipital_RF_acc,Frontal_Left Frontal_RF_acc,Frontal_Right Frontal_RF_acc,Frontal_Central_RF_acc,Frontal_Left Central_RF_acc,Frontal_Right Central_RF_acc,Frontal_Left Temporal_RF_acc,Frontal_Right Temporal_RF_acc,Frontal_Parietal_RF_acc,Frontal_Left Parietal_RF_acc,Frontal_Right Parietal_RF_acc,Frontal_Occipital_RF_acc,Left Frontal_Right Frontal_RF_acc,Left Frontal_Central_RF_acc,Left Frontal_Left Central_RF_acc,Left Frontal_Right Central_RF_acc,Left Frontal_Left Temporal_RF_acc,Left Frontal_Right Temporal_RF_acc,Left Frontal_Parietal_RF_acc,Left Frontal_Left Parietal_RF_acc,Left Frontal_Right Parietal_RF_acc,Left Frontal_Occipital_RF_acc,Right Frontal_Central_RF_acc,Right Frontal_Left Central_RF_acc,Right Frontal_Right Central_RF_acc,Right Frontal_Left Temporal_RF_acc,Right Frontal_Right Temporal_RF_acc,Right Frontal_Parietal_RF_acc,Right Frontal_Left Parietal_RF_acc,Right Frontal_Right Parietal_RF_acc,Right Frontal_Occipital_RF_acc,Central_Left Central_RF_acc,Central_Right Central_RF_acc,Central_Left Temporal_RF_acc,Central_Right Temporal_RF_acc,Central_Parietal_RF_acc,Central_Left Parietal_RF_acc,Central_Right Parietal_RF_acc,Central_Occipital_RF_acc,Left Central_Right Central_RF_acc,Left Central_Left Temporal_RF_acc,Left Central_Right Temporal_RF_acc,Left Central_Parietal_RF_acc,Left Central_Left Parietal_RF_acc,Left Central_Right Parietal_RF_acc,Left Central_Occipital_RF_acc,Right Central_Left Temporal_RF_acc,Right Central_Right Temporal_RF_acc,Right Central_Parietal_RF_acc,Right Central_Left Parietal_RF_acc,Right Central_Right Parietal_RF_acc,Right Central_Occipital_RF_acc,Left Temporal_Right Temporal_RF_acc,Left Temporal_Parietal_RF_acc,Left Temporal_Left Parietal_RF_acc,Left Temporal_Right Parietal_RF_acc,Left Temporal_Occipital_RF_acc,Right Temporal_Parietal_RF_acc,Right Temporal_Left Parietal_RF_acc,Right Temporal_Right Parietal_RF_acc,Right Temporal_Occipital_RF_acc,Parietal_Left Parietal_RF_acc,Parietal_Right Parietal_RF_acc,Parietal_Occipital_RF_acc,Left Parietal_Right Parietal_RF_acc,Left Parietal_Occipital_RF_acc,Right Parietal_Occipital_RF_acc
0,0.409091,0.394697,0.411364,0.375758,0.409091,0.437879,0.412121,0.421212,0.368182,0.356061,0.418939,0.339394,0.356061,0.284848,0.266667,0.239394,0.302273,0.211364,0.292424,0.256818,0.228788,0.32803,0.212121,0.292424,0.275758,0.259091,0.275758,0.312879,0.337879,0.332576,0.319697,0.372727,0.302273,0.268939,0.22197,0.231061,0.231818,0.25,0.251515,0.256061,0.302273,0.232576,0.195455,0.19697,0.187879,0.178788,0.231061,0.178788,0.186364,0.152273,0.212879,0.151515,0.150758,0.168182,0.194697,0.194697,0.159091,0.212121,0.240152,0.177273,0.204545,0.255303,0.186364,0.187121,0.17803,0.150758,0.210606,0.133333,0.214394,0.238636,0.248485,0.203788,0.17803,0.231818,0.143182,0.253788,0.159848,0.166667


,Prefrontal_Frontal_DT_acc,Prefrontal_Left Frontal_DT_acc,Prefrontal_Right Frontal_DT_acc,Prefrontal_Central_DT_acc,Prefrontal_Left Central_DT_acc,Prefrontal_Right Central_DT_acc,Prefrontal_Left Temporal_DT_acc,Prefrontal_Right Temporal_DT_acc,Prefrontal_Parietal_DT_acc,Prefrontal_Left Parietal_DT_acc,Prefrontal_Right Parietal_DT_acc,Prefrontal_Occipital_DT_acc,Frontal_Left Frontal_DT_acc,Frontal_Right Frontal_DT_acc,Frontal_Central_DT_acc,Frontal_Left Central_DT_acc,Frontal_Right Central_DT_acc,Frontal_Left Temporal_DT_acc,Frontal_Right Temporal_DT_acc,Frontal_Parietal_DT_acc,Frontal_Left Parietal_DT_acc,Frontal_Right Parietal_DT_acc,Frontal_Occipital_DT_acc,Left Frontal_Right Frontal_DT_acc,Left Frontal_Central_DT_acc,Left Frontal_Left Central_DT_acc,Left Frontal_Right Central_DT_acc,Left Frontal_Left Temporal_DT_acc,Left Frontal_Right Temporal_DT_acc,Left Frontal_Parietal_DT_acc,Left Frontal_Left Parietal_DT_acc,Left Frontal_Right Parietal_DT_acc,Left Frontal_Occipital_DT_acc,Right Frontal_Central_DT_acc,Right Frontal_Left Central_DT_acc,Right Frontal_Right Central_DT_acc,Right Frontal_Left Temporal_DT_acc,Right Frontal_Right Temporal_DT_acc,Right Frontal_Parietal_DT_acc,Right Frontal_Left Parietal_DT_acc,Right Frontal_Right Parietal_DT_acc,Right Frontal_Occipital_DT_acc,Central_Left Central_DT_acc,Central_Right Central_DT_acc,Central_Left Temporal_DT_acc,Central_Right Temporal_DT_acc,Central_Parietal_DT_acc,Central_Left Parietal_DT_acc,Central_Right Parietal_DT_acc,Central_Occipital_DT_acc,Left Central_Right Central_DT_acc,Left Central_Left Temporal_DT_acc,Left Central_Right Temporal_DT_acc,Left Central_Parietal_DT_acc,Left Central_Left Parietal_DT_acc,Left Central_Right Parietal_DT_acc,Left Central_Occipital_DT_acc,Right Central_Left Temporal_DT_acc,Right Central_Right Temporal_DT_acc,Right Central_Parietal_DT_acc,Right Central_Left Parietal_DT_acc,Right Central_Right Parietal_DT_acc,Right Central_Occipital_DT_acc,Left Temporal_Right Temporal_DT_acc,Left Temporal_Parietal_DT_acc,Left Temporal_Left Parietal_DT_acc,Left Temporal_Right Parietal_DT_acc,Left Temporal_Occipital_DT_acc,Right Temporal_Parietal_DT_acc,Right Temporal_Left Parietal_DT_acc,Right Temporal_Right Parietal_DT_acc,Right Temporal_Occipital_DT_acc,Parietal_Left Parietal_DT_acc,Parietal_Right Parietal_DT_acc,Parietal_Occipital_DT_acc,Left Parietal_Right Parietal_DT_acc,Left Parietal_Occipital_DT_acc,Right Parietal_Occipital_DT_acc
0,0.331818,0.385606,0.356061,0.349242,0.366667,0.385606,0.431061,0.32197,0.412879,0.374242,0.40303,0.287121,0.32803,0.294697,0.242424,0.240909,0.267424,0.291667,0.284848,0.27803,0.275,0.292424,0.213636,0.275758,0.229545,0.195455,0.213636,0.205303,0.276515,0.250758,0.212879,0.319697,0.284848,0.243182,0.268182,0.312121,0.222727,0.214394,0.231818,0.214394,0.260606,0.2,0.205303,0.250758,0.212879,0.232576,0.223485,0.205303,0.238636,0.197727,0.238636,0.212879,0.231818,0.185606,0.168939,0.247727,0.212121,0.291667,0.275758,0.194697,0.17803,0.24697,0.186364,0.267424,0.259091,0.187879,0.293939,0.186364,0.270455,0.178788,0.275,0.27803,0.224242,0.250758,0.188636,0.231818,0.240152,0.258333


,Prefrontal_Frontal_Ada_B_acc,Prefrontal_Left Frontal_Ada_B_acc,Prefrontal_Right Frontal_Ada_B_acc,Prefrontal_Central_Ada_B_acc,Prefrontal_Left Central_Ada_B_acc,Prefrontal_Right Central_Ada_B_acc,Prefrontal_Left Temporal_Ada_B_acc,Prefrontal_Right Temporal_Ada_B_acc,Prefrontal_Parietal_Ada_B_acc,Prefrontal_Left Parietal_Ada_B_acc,Prefrontal_Right Parietal_Ada_B_acc,Prefrontal_Occipital_Ada_B_acc,Frontal_Left Frontal_Ada_B_acc,Frontal_Right Frontal_Ada_B_acc,Frontal_Central_Ada_B_acc,Frontal_Left Central_Ada_B_acc,Frontal_Right Central_Ada_B_acc,Frontal_Left Temporal_Ada_B_acc,Frontal_Right Temporal_Ada_B_acc,Frontal_Parietal_Ada_B_acc,Frontal_Left Parietal_Ada_B_acc,Frontal_Right Parietal_Ada_B_acc,Frontal_Occipital_Ada_B_acc,Left Frontal_Right Frontal_Ada_B_acc,Left Frontal_Central_Ada_B_acc,Left Frontal_Left Central_Ada_B_acc,Left Frontal_Right Central_Ada_B_acc,Left Frontal_Left Temporal_Ada_B_acc,Left Frontal_Right Temporal_Ada_B_acc,Left Frontal_Parietal_Ada_B_acc,Left Frontal_Left Parietal_Ada_B_acc,Left Frontal_Right Parietal_Ada_B_acc,Left Frontal_Occipital_Ada_B_acc,Right Frontal_Central_Ada_B_acc,Right Frontal_Left Central_Ada_B_acc,Right Frontal_Right Central_Ada_B_acc,Right Frontal_Left Temporal_Ada_B_acc,Right Frontal_Right Temporal_Ada_B_acc,Right Frontal_Parietal_Ada_B_acc,Right Frontal_Left Parietal_Ada_B_acc,Right Frontal_Right Parietal_Ada_B_acc,Right Frontal_Occipital_Ada_B_acc,Central_Left Central_Ada_B_acc,Central_Right Central_Ada_B_acc,Central_Left Temporal_Ada_B_acc,Central_Right Temporal_Ada_B_acc,Central_Parietal_Ada_B_acc,Central_Left Parietal_Ada_B_acc,Central_Right Parietal_Ada_B_acc,Central_Occipital_Ada_B_acc,Left Central_Right Central_Ada_B_acc,Left Central_Left Temporal_Ada_B_acc,Left Central_Right Temporal_Ada_B_acc,Left Central_Parietal_Ada_B_acc,Left Central_Left Parietal_Ada_B_acc,Left Central_Right Parietal_Ada_B_acc,Left Central_Occipital_Ada_B_acc,Right Central_Left Temporal_Ada_B_acc,Right Central_Right Temporal_Ada_B_acc,Right Central_Parietal_Ada_B_acc,Right Central_Left Parietal_Ada_B_acc,Right Central_Right Parietal_Ada_B_acc,Right Central_Occipital_Ada_B_acc,Left Temporal_Right Temporal_Ada_B_acc,Left Temporal_Parietal_Ada_B_acc,Left Temporal_Left Parietal_Ada_B_acc,Left Temporal_Right Parietal_Ada_B_acc,Left Temporal_Occipital_Ada_B_acc,Right Temporal_Parietal_Ada_B_acc,Right Temporal_Left Parietal_Ada_B_acc,Right Temporal_Right Parietal_Ada_B_acc,Right Temporal_Occipital_Ada_B_acc,Parietal_Left Parietal_Ada_B_acc,Parietal_Right Parietal_Ada_B_acc,Parietal_Occipital_Ada_B_acc,Left Parietal_Right Parietal_Ada_B_acc,Left Parietal_Occipital_Ada_B_acc,Right Parietal_Occipital_Ada_B_acc
0,0.276515,0.354545,0.291667,0.383333,0.344697,0.329545,0.338636,0.347727,0.365909,0.305303,0.391667,0.408333,0.384848,0.312121,0.242424,0.274242,0.285606,0.284091,0.265152,0.285606,0.257576,0.293939,0.222727,0.367424,0.329545,0.293939,0.263636,0.281061,0.312879,0.234091,0.276515,0.353788,0.240909,0.234091,0.215152,0.275,0.259091,0.17803,0.268182,0.214394,0.275,0.241667,0.268939,0.279545,0.259091,0.275758,0.232576,0.231061,0.301515,0.196212,0.254545,0.276515,0.241667,0.204545,0.309848,0.250758,0.229545,0.238636,0.231061,0.213636,0.247727,0.249242,0.301515,0.275,0.277273,0.248485,0.312879,0.275,0.231061,0.275758,0.339394,0.259091,0.207576,0.276515,0.249242,0.22197,0.17803,0.335606


120.08914113044739


In [70]:
display(pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'RF' in col]].mean(axis = 0)).max())
display(pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'DT' in col]].mean(axis = 0)).max())
display(pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'Ada_B' in col]].mean(axis = 0)).max())

0    0.437879
dtype: float64

0    0.431061
dtype: float64

0    0.408333
dtype: float64

### Using all data

In [48]:
#Now X, Y and groups have been loaded for N1 data - generate X,y groups for a specific experiment
#1.Generate expt specific X,y,groups
t1 = time.time()

X_expt , y_expt , groups_expt, expt_info = run_expts.generate_expt_x_y_groups(X,y,groups, 4 )
results_df = run_expts.run_mv_tsc(X_expt,y_expt,groups_expt, {'RF' : RandomForestClassifier(random_state = 5) , 'DT' : DecisionTreeClassifier() , 'Ada_B' : AdaBoostClassifier(random_state = 5)} , return_df = True , subset_names_and_cols = all_data_dict, random_states = [1,2] )
res_df = run_expts.generate_subset_acc_std(results_df, return_df = True)

display(pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'RF' in col]].mean(axis = 0)).T)
display(pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'DT' in col]].mean(axis = 0)).T)
display(pd.DataFrame(res_df[[col for col in res_df.columns if 'acc' in col and 'Ada_B' in col]].mean(axis = 0)).T)


t2 = time.time()
time_taken = t2-t1
print(time_taken)

,All_regions_RF_acc,All_regions_2_RF_acc
0,0.275,0.275


,All_regions_DT_acc,All_regions_2_DT_acc
0,0.358333,0.331818


,All_regions_Ada_B_acc,All_regions_2_Ada_B_acc
0,0.327273,0.327273


3.95713472366333
